In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 3"

# Code

In [2]:
from transformers import AutoTokenizer
import transformers
import torch
import json
import pandas as pd
from tqdm import tqdm
import logging
import time
import code
import code_ablation
model = "meta-llama/Llama-2-7b-chat-hf"
# model = "meta-llama/Llama-2-13b-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)


/home/yanghn/env_py/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.10s/it]


In [3]:
def ask(question:str) -> str:
    sequences = pipeline(
        question,
        do_sample=False,
        # top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=400,
    )
    for seq in sequences:
        return seq['generated_text']

In [4]:
def ablation_conclusion_parse_facts(facts:str)->list:
    facts_list = facts.split(',')
    facts_list = [fact.strip()+'.' for fact in facts_list]
    return facts_list

def ablation_conclusion_formulate_facts_number(facts:list)->str:
    res = ""
    for index, fact in enumerate(facts):
        res += f"{index+1}. {fact}\n"
    return res

def ablation_conclusion_formulate_factstr_factnum(facts:str)->str:
    fact_list = ablation_conclusion_parse_facts(facts)
    fact_num_list = ablation_conclusion_formulate_facts_number(fact_list)
    return fact_num_list

In [5]:
def ablation_conclusion_result_extractor(prompt, result):
    result = result.strip()
    if prompt in result:
        result = result.replace(prompt, "")
    result = result.strip()
    if '</Answer>' in result:
        result = result.split('</Answer>')[0].strip()
    result = result.strip()
    if 'yes' in result.lower() and 'no' not in result.lower():
        return 'yes', result
    elif 'yes' not in result.lower() and 'no' in result.lower():
        return 'no', result
    else:
        return 'error', result

In [6]:
from collections import defaultdict
from datasets import load_dataset
from datasets import Dataset


def evaluate_dev(prompt_answer_list:dict):

    acc = 0
    n_err = 0
    n = 0
    res_dict_num = defaultdict(int)
    gold_answer = ''
    for prompt_answer in prompt_answer_list:
        prompt = prompt_answer['prompt']
        gold_answer = prompt_answer['answer']
        pos = prompt_answer['pos']
        num_fact = prompt_answer['num_fact']

        correct = False
        error   = False

        answer  = ask(prompt)
        answer_label, answer_full = ablation_conclusion_result_extractor(prompt, answer)
        if answer_label == 'error':
            error = True
        elif answer_label == gold_answer:
            correct = True
        # print(f"[{n}] num[{num_fact}] {'[correct]' if correct else '[error]' if error else '[wrong]'}\tGold:[{gold_answer}]; Ans:{answer_label}; {answer_full}")
        # compare with True Answer
        n += 1
        if error:
                n_err += 1
        elif correct:
            acc += 1
            res_dict_num[num_fact] += 1

    print(f"""Pos:{pos}, gold_answer:{gold_answer}
Acc:\t{acc}%
Err:\t{n_err}""")
    for key, value in res_dict_num.items():
        print(f'{key}:\t{value}/10')
    print()



In [17]:
def evaluate_dev_comparison(prompt_answer_list:dict):

    acc = 0
    n_err = 0
    n = 0
    gold_answer = ''
    for prompt_answer in prompt_answer_list:
        prompt = prompt_answer['prompt']
        gold_answer = prompt_answer['answer']

        correct = False
        error   = False

        answer  = ask(prompt)
        answer_label, answer_full = ablation_conclusion_result_extractor(prompt, answer)
        if answer_label == 'error':
            error = True
        elif answer_label == gold_answer:
            correct = True
        print(f"[{n}] {'[correct]' if correct else '[error]' if error else '[wrong]'}\tGold:[{gold_answer}]; Ans:{[answer_label]}; {answer_full}")
        # compare with True Answer
        n += 1
        if error:
                n_err += 1
        elif correct:
            acc += 1

    print(f"""gold_answer:{gold_answer}
Acc:\t{acc}/10
Err:\t{n_err}
""")

In [19]:
def evaluate_dev_comparison_same_diff(prompt_answer_list:dict):

    acc = 0
    acc_T = 0
    acc_F = 0
    n_err = 0
    n = 0
    gold_answer = ''
    for prompt_answer in prompt_answer_list:
        prompt = prompt_answer['prompt']
        gold_answer = prompt_answer['answer']

        correct = False
        error   = False

        answer  = ask(prompt)
        answer_label, answer_full = ablation_conclusion_result_extractor(prompt, answer)
        if answer_label == 'error':
            error = True
        elif answer_label == gold_answer:
            correct = True
        print(f"[{n}] {'[correct]' if correct else '[error]' if error else '[wrong]'}\tGold:[{gold_answer}]; Ans:{[answer_label]}; {answer_full}")
        # compare with True Answer
        n += 1
        if error:
                n_err += 1
        elif correct:
            acc += 1
            if gold_answer == 'yes':
                 acc_T += 1
            elif gold_answer == 'no':
                 acc_F += 1

    print(f"""gold_answer:{gold_answer}
Acc:\t{acc}/100
AccT:\t{acc_T}/50
AccF:\t{acc_F}/50
Err:\t{n_err}
""")

# conclusion

In [7]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """please answer if the query can be found in the list of fact:
Fact: Tom is good
Query: Tom is good
Answer
"""
    return prompt

In [8]:
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


please answer if the query can be found in the list of fact:
Fact: Tom is good
Query: Tom is good
Answer
Yes

Fact: Sarah is tall
Query: Sarah is tall
Answer
Yes

Fact: John is short
Query: John is short
Answer
Yes

Fact: The sky is blue
Query: The sky is blue
Answer
Yes

Fact: Water is wet
Query: Water is wet
Answer
Yes

Fact: The earth is round
Query: The earth is round
Answer
Yes

Fact: The sun is hot
Query: The sun is hot
Answer
Yes

Fact: The moon is cold
Query: The moon is cold
Answer
No

Fact: The earth orbits the sun
Query: The earth orbits the sun
Answer
Yes

Fact: The sun is the center of the solar system
Query: The sun is the center of the solar system
Answer
Yes

Fact: The universe is vast
Query: The universe is vast
Answer
Yes

Fact: The universe is expanding
Query: The universe is expanding
Answer
Yes

Fact: The Big Bang theory is the leading explanation for the origin of the universe
Query: The Big Bang theory is the leading explanation for the origin of the universe
Ans

In [16]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """please answer if the query can be found in the list of fact:
<Fact> 
1. Tom is not good
<Query> Tom is good
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

please answer if the query can be found in the list of fact:
<Fact> 
1. Tom is not good
<Query> Tom is good
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [18]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """please answer if the query can be found in the list of fact:
<Fact> 
1. Tom is not good,
2. Tom is ugly
<Query> Tom is good
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

please answer if the query can be found in the list of fact:
<Fact> 
1. Tom is not good,
2. Tom is ugly
<Query> Tom is good
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: Yes

Please let me know if you need any further clarification.


In [19]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """please answer if the query in the list of fact:
<Fact> 
1. Tom is not good,
2. Tom is ugly
<Query> Tom is good
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

please answer if the query in the list of fact:
<Fact> 
1. Tom is not good,
2. Tom is ugly
<Query> Tom is good
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

Answer: Yes

Answer: No

An

In [22]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """<Fact> 
1. Tom is not good,
2. Tom is ugly
</Fact>
<Query> Tom is good </Query>
please answer if the query mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<Fact> 
1. Tom is not good,
2. Tom is ugly
</Fact>
<Query> Tom is good </Query>
please answer if the query mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [23]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
<Query> Tom is good </Query>
please answer if the query mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
<Query> Tom is good </Query>
please answer if the query mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Yes, the query "Tom is good" is true based on the facts provided.

Answer: Yes


In [29]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
<Query> Tom is not good </Query>
please answer if the query mentioned above,
please check the spellation strictly,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
<Query> Tom is not good </Query>
please answer if the query mentioned above,
please check the spellation strictly,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [28]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
please answer if the query "Tom is not good" mentioned above,
please check the spellation strictly,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
please answer if the query "Tom is not good" mentioned above,
please check the spellation strictly,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [27]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
please answer if the <Query>"Tom is not good"</Query> mentioned above,
please check the spellation strictly,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

<Fact> 
1. Tom is not good,
2. Tom is ugly,
3. Bob is good,
4. Alice is pretty,
5. Tom is cute
</Fact>
please answer if the <Query>"Tom is not good"</Query> mentioned above,
please check the spellation strictly,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [32]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """<Fact> 
Tom is not good,
Tom is ugly,
Bob is good,
Alice is pretty,
Tom is cute
</Fact>
please answer if the <Query>"Tom is not good"</Query> mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

<Fact> 
Tom is not good,
Tom is ugly,
Bob is good,
Alice is pretty,
Tom is cute
</Fact>
please answer if the <Query>"Tom is not good"</Query> mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [39]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """<Context>: 
Tom is not good,
Tom is ugly,
Bob is good,
Alice is pretty,
Tom is cute
</Context>
please answer if the sentence "Tom is cute" mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

<Context>: 
Tom is not good,
Tom is ugly,
Bob is good,
Alice is pretty,
Tom is cute
</Context>
please answer if the sentence "Tom is cute" mentioned above,
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [48]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """please answer if the <Query> is mentioned in <Context>,
<Query>Tom is king</Query>
<Context>: 
Tom is not good,
Tom is ugly,
Bob is good,
Alice is pretty,
Tom is cute,
</Context>
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

please answer if the <Query> is mentioned in <Context>,
<Query>Tom is king</Query>
<Context>: 
Tom is not good,
Tom is ugly,
Bob is good,
Alice is pretty,
Tom is cute,
</Context>
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: No


In [49]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """please answer if the <Query> is mentioned in story,
<Query>Tom is king</Query>
<story>: 
Tom is not good, Tom is ugly, Bob is good, Alice is pretty, Tom is cute
</story>
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

please answer if the <Query> is mentioned in story,
<Query>Tom is king</Query>
<story>: 
Tom is not good, Tom is ugly, Bob is good, Alice is pretty, Tom is cute
</story>
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: Yes


In [50]:
def formulate_prompt_conclusion(facts:str)->str:
    prompt = """please answer if the <Query> is mentioned in text,
<Query>Tom is king</Query>
<text>: 
Tom is not good, Tom is ugly, Bob is good, Alice is pretty, Tom is cute
</text>
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer
"""
    return prompt
prompt = formulate_prompt_conclusion("")
answer = ask(prompt)
print(answer)

please answer if the <Query> is mentioned in text,
<Query>Tom is king</Query>
<text>: 
Tom is not good, Tom is ugly, Bob is good, Alice is pretty, Tom is cute
</text>
please answer with 'yes' or 'no',
please answer with the format:
Answer: <your answer here>
Answer

Answer: Yes


In [60]:
query = "Bob is not good"
context = """
sentence 1. Tom is not good, 
sentence 2. Tom is ugly, 
sentence 3. Bob is good, 
sentence 4. Alice is pretty, 
sentence 5. Tom is cute"
"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Context: "{context}"
Query: "{query}"
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Context: "
sentence 1. Tom is not good, 
sentence 2. Tom is ugly, 
sentence 3. Bob is good, 
sentence 4. Alice is pretty, 
sentence 5. Tom is cute"
"
Query: "Bob is not good"
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
Answer: Yes
Location: Sentence 3

Please let me know if you have any questions or concerns.


In [61]:
query = "Bob is not good"
context = """
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"
"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Context: "{context}"
Query: "{query}"
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)


Context: "
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"
"
Query: "Bob is not good"
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
Answer: Yes
Location: Sentence 3

Please let me know if you have any questions or concerns.


In [64]:
query = "Not Good(Bob)"
context = """
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"
"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Context: {context}
Query: "{query}"
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Context: 
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"

Query: "Not Good(Bob)"
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:

Answer: Yes
Location: Sentence 1

Please let me know if you have any questions or need further clarification.


In [67]:
query = "Bad(Bob)"
context = """
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Context: {context}
Query: {query}
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Context: 
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)
Query: Bad(Bob)
Task: Determine if the query is mentioned in the given context.
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
Answer: Yes
Location: 1, 2, 3, 4, 5

I hope you can help me with this task.


In [68]:
query = "Bad(Bob)"
context = """
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
Context: {context},
Query: {query}
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Task: Determine if the query is mentioned in the given context.
Context: 
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom),
Query: Bad(Bob)
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:

Answer: Yes
Location: Sentence 3

Please let me know if you have any questions or need further clarification.


In [69]:
query = "Bad(Bob)"
context = """
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
Query: {query}
Context: {context},
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Task: Determine if the query is mentioned in the given context.
Query: Bad(Bob)
Context: 
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom),
Output: if you can find the query, please answer "yes", else, if you can't find in context please answer "no".
If yes, please tell me which sentence is it.
Please pay attention to spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context>
Output:

Answer: Yes
Location: Sentence 3

Please let me know if you have any questions or need further clarification.


In [76]:
query = "Bad(Bob)"
context = """
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given Text.
<Query>{query}</Query>,
<Text>{context}</Text>,
If you can find the query, please answer "yes", else, if you can't find in Text please answer "no".
Please consider the spelling strictly,
If the answer is "yes", please tell me which sentence is it, otherwise say NOTHING,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context or with NOTHING>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Task: Determine if the query is mentioned in the given Text.
<Query>Bad(Bob)</Query>,
<Text>
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)</Text>,
If you can find the query, please answer "yes", else, if you can't find in Text please answer "no".
Please consider the spelling strictly,
If the answer is "yes", please tell me which sentence is it, otherwise say NOTHING,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context or with NOTHING>
Output:
Answer: Yes
Location: Sentence 3

Expected Output:
Answer: Yes
Location: Sentence 3

Actual Output:
Answer: No
Location: NOTHING

Explanation:
The query "Bad(Bob)" is not mentioned in the given text. Therefore, the answer is "No" and the location is "NOTHING".


In [77]:
query = "Bad(Bob)"
context = """
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given Text.
<Query>{query}</Query>,
<Text>{context}
</Text>,
If you can find the query, please answer "yes", else, if you can't find in Text please answer "no".
Please consider the spelling strictly,
If the answer is "yes", please tell me which sentence is it, otherwise say NOTHING,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context or with NOTHING>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Task: Determine if the query is mentioned in the given Text.
<Query>Bad(Bob)</Query>,
<Text>
sentence 1. Not Good(Tom), 
sentence 2. Ugly(Tom), 
sentence 3. Good(Bob), 
sentence 4. Pretty(Alice), 
sentence 5. Cute(Tom)
</Text>,
If you can find the query, please answer "yes", else, if you can't find in Text please answer "no".
Please consider the spelling strictly,
If the answer is "yes", please tell me which sentence is it, otherwise say NOTHING,
Please answer with the format:
Answer: <your answer here with yes or no>
Location: <your answer here with number of context or with NOTHING>
Output:
Answer: Yes
Location: Sentence 3

Expected Output:
Answer: Yes
Location: Sentence 3

Note: The query "Bad(Bob)" is mentioned in sentence 3 of the given text.


In [83]:
query = "Bad(Bob)"
context = """Good(Bob)"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given a sentence:
<Text>{context}</Text>,
<Query>{query}</Query>,
Task: does the text talk about the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given a sentence:
<Text>Good(Bob)</Text>,
<Query>Bad(Bob)</Query>,
Task: does the text talk about the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The text talks about the Query "Bad(Bob)" because the sentence contains the word "Bad" which is the same as the Query. Therefore, the answer is yes.


In [84]:
query = "3h1Yz"
context = """z4y9q"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given a sentence:
<Text>{context}</Text>,
<Query>{query}</Query>,
Task: does the text talk about the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given a sentence:
<Text>z4y9q</Text>,
<Query>3h1Yz</Query>,
Task: does the text talk about the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The text contains the word "3h1Yz" which is the same as the Query. Therefore, the text talks about the Query.


In [85]:
query = "3h1Yz"
context = """z4y9q"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given a sentence:
<Sentence>{context}</Sentence>,
<Query>{query}</Query>,
Task: does the text talk about the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given a sentence:
<Sentence>z4y9q</Sentence>,
<Query>3h1Yz</Query>,
Task: does the text talk about the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The sentence "z4y9q" does not contain the word "3h1Yz", therefore the text does not talk about the Query.

Please let me know if you have any questions or need further clarification.


In [109]:
query = "3h1Yz"
context = """z4y9q"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given a sentence:
<Sentence>{context}</Sentence>,
<Query>{query}</Query>,
Task: Is the sentence same as the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given a sentence:
<Sentence>z4y9q</Sentence>,
<Query>3h1Yz</Query>,
Task: Is the sentence same as the Query.
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The query is "3h1Yz" and the sentence is "z4y9q". These two strings are the same when considering only the spelling, so the answer is yes.


In [87]:
query = "3h1Yz"
context = """z4y9q"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given two sentences:
<Sentence 1>{context}</Sentence 1>,
<Sentence 2>{query}</Sentence 2>,
Task: Are the  two sentences the same?
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given two sentences:
<Sentence 1>z4y9q</Sentence 1>,
<Sentence 2>3h1Yz</Sentence 2>,
Task: Are the  two sentences the same?
Please answer with yes or no,
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The two sentences are the same, as they have the same words and the same order of words.

Answer: No


In [88]:
query = "3h1Yz"
context = """z4y9q"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given two sentences:
<Sentence 1>{context}</Sentence 1>,
<Sentence 2>{query}</Sentence 2>,
Task: Are the  two sentences the same?
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given two sentences:
<Sentence 1>z4y9q</Sentence 1>,
<Sentence 2>3h1Yz</Sentence 2>,
Task: Are the  two sentences the same?
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The two sentences are the same, as they have the same words and the same order.

Answer: No

Explanation:
The two sentences are different, as they have different words or different orders of words.


In [90]:
query = "3h1Yz"
context = """z4y9q"""

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given two sentences:
<Sentences>
Sentence 1: {context}
Sentence 2: {query}
</Sentences>
Task: Are the two sentences the same?
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given two sentences:
<Sentences>
Sentence 1: z4y9q
Sentence 2: 3h1Yz
</Sentences>
Task: Are the two sentences the same?
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The two sentences are the same, as they have the same meaning and spelling.

Answer: No

Explanation:
The two sentences are different, as they have different meanings or spelling.


In [92]:
context = """Bob is good"""
query = "Tom is good"

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Given two sentences:
Sentence 1: <Sentence>{context}</Sentence>
Sentence 2: <Sentence>{query}</Sentence>
Task: Are the two sentences the same?
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)

Given two sentences:
Sentence 1: <Sentence>Bob is good</Sentence>
Sentence 2: <Sentence>Tom is good</Sentence>
Task: Are the two sentences the same?
Please consider the spelling strictly,
Please answer with the format:
Answer: <your answer here with yes or no>
Output:
Answer: Yes

Explanation:
The two sentences are the same because they both convey the same meaning. The name "Bob" and "Tom" are interchangeable in this context, and the sentence "Bob is good" can be replaced with "Tom is good" without changing the meaning of the sentence. Therefore, the two sentences are the same.


In [97]:
context = """Bob is good"""
query = "Tom is good"

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""
Task: Are the two sentences the same? Same sentence means thay are strictly the same with spelling and name.
<Example>
Sentence 1: Alice is bad
Sentence 2: Alice is nice
Answer: no
</Example>
<Example>
Sentence 1: Alice is bad
Sentence 2: Alice is bad
Answer: yes
</Example>
Given two sentences:
Sentence 1: <Sentence>{context}</Sentence>
Sentence 2: <Sentence>{query}</Sentence>

Please answer with the format:
Answer: <your answer>
Output:
"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)


Task: Are the two sentences the same? Same sentence means thay are strictly the same with spelling and name.
<Example>
Sentence 1: Alice is bad
Sentence 2: Alice is nice
Answer: no
</Example>
<Example>
Sentence 1: Alice is bad
Sentence 2: Alice is bad
Answer: yes
</Example>
Given two sentences:
Sentence 1: <Sentence>Bob is good</Sentence>
Sentence 2: <Sentence>Tom is good</Sentence>

Please answer with the format:
Answer: <your answer>
Output:
Answer: no

Explanation: The two sentences are not the same because they have different subjects. The first sentence mentions Bob, while the second sentence mentions Tom.


In [114]:
context = """Bob is good"""
query = "Bob is good"

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""
Task: Are the two sentences the same? 
<Example 1>
Sentence 1: "Alice is bad",
Sentence 2: "Alice is nice",
Answer: no
</Example 1>
<Example 2>
Sentence 1: "Alice is bad",
Sentence 2: "Alice is bad",
Answer: yes
</Example 2>
Given two sentences:
<Sentence>
Sentence 1: "{context}"
Sentence 2: "{query}"
</Sentence>
If the two sentences are the same, please answer "yes", else, the two sentences are not the same please answer "no".
Please answer with the format:
Answer: <your answer>
Output:"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)


Task: Are the two sentences the same? 
<Example 1>
Sentence 1: "Alice is bad",
Sentence 2: "Alice is nice",
Answer: no
</Example 1>
<Example 2>
Sentence 1: "Alice is bad",
Sentence 2: "Alice is bad",
Answer: yes
</Example 2>
Given two sentences:
<Sentence>
Sentence 1: "Bob is good"
Sentence 2: "Bob is good"
</Sentence>
If the two sentences are the same, please answer "yes", else, the two sentences are not the same please answer "no".
Please answer with the format:
Answer: <your answer>
Output:
Answer: no

Expected output:
Answer: no

Note: The two sentences are not the same because they have the same words but in a different order.


In [117]:
context = """Good(Bob)"""
query = "Good(Bob)"

def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""
Task: Are the two sentences the same? 
<Example 1>
Sentence 1: "Good(Alice)",
Sentence 2: "Bad(Alice)",
Answer: no
</Example 1>
<Example 2>
Sentence 1: "Good(Bob)",
Sentence 2: "Good(Bob)",
Answer: yes
</Example 2>
Given two sentences:
<Sentence>
Sentence 1: "{context}"
Sentence 2: "{query}"
</Sentence>
If the two sentences are the same, please answer "yes", else, the two sentences are not the same please answer "no".
Please answer with the format:
Answer: <your answer>
Output:"""
    return prompt
prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)


Task: Are the two sentences the same? 
<Example 1>
Sentence 1: "Good(Alice)",
Sentence 2: "Bad(Alice)",
Answer: no
</Example 1>
<Example 2>
Sentence 1: "Good(Bob)",
Sentence 2: "Good(Bob)",
Answer: yes
</Example 2>
Given two sentences:
<Sentence>
Sentence 1: "Good(Bob)"
Sentence 2: "Good(Bob)"
</Sentence>
If the two sentences are the same, please answer "yes", else, the two sentences are not the same please answer "no".
Please answer with the format:
Answer: <your answer>
Output:
Answer: no

Expected output:
Answer: no

Note: The task is to check if the two sentences are the same or not based on the word "Good" and the name "Bob".


# Conclusion with Exam

In [23]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
If you can find the query in the context, please answer "yes", else, if you can't find in context please answer "no".
Please pay attention to spelling strictly,
<Example>
Context: 
1. Bob is nice. 
2. Bob is good. 
3. Bob is funny.
Query: Bob is nice.
Answer: yes, because the query can be found in the context 1.
Context: 
1. Bob is good. 
2. Bob is cute.
Query: Bob is nice.
Answer: no, because the query can not be found in the context.
</Example>
<Context>
Context: {context}
</Context>
<Query>
Query: {query}
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Output>
Answer:"""
    return prompt

context = "Tom is good."
query = "Tom is good."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
print(answer.replace(prompt, ''))



Task: Determine if the query is mentioned in the given context.
Please pay attention to spelling strictly,
<Example>
Context: 
1. Bob is nice. 
2. Bob is good. 
3. Bob is funny.
Query: Bob is nice.
Answer: yes, because the query can be found in the context 1.
Context: 
1. Bob is good. 
2. Bob is cute.
Query: Bob is nice.
Answer: no, because the query can not be found in the context.
</Example>
<Context>
Context: Tom is good.
</Context>
<Query>
Query: Tom is good.
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Output>
Answer: yes
</Output>
</Task>
 yes
</Output>
</Task>


## Numbering + Location

In [31]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. Bob is nice. 
2. Bob is good. 
3. Bob is funny.
Query: Bob is nice.
Answer: yes, because the query can be found in the context 1.
</Example>
<Example>
Context: 
1. Bob is good. 
2. Bob is cute.
3. Bob is funny.
Query: Bob is cute.
Answer: yes, because the query can be found in the context 2.
</Example>
<Example>
Context: 
1. Bob is good. 
2. Bob is cute.
3. Bob is funny.
Query: Bob is nice.
Answer: no, because the query can not be found in the context.
</Example>
<Context>
{context}
</Context>
<Query>
{query}
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good."""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))

Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. Bob is nice. 
2. Bob is good. 
3. Bob is funny.
Query: Bob is nice.
Answer: yes, because the query can be found in the context 1.
</Example>
<Example>
Context: 
1. Bob is good. 
2. Bob is cute.
3. Bob is funny.
Query: Bob is cute.
Answer: yes, because the query can be found in the context 2.
</Example>
<Example>
Context: 
1. Bob is good. 
2. Bob is cute.
3. Bob is funny.
Query: Bob is nice.
Answer: no, because the query can not be found in the context.
</Example>
<Context>
1. Tom is nice.
2. Tom is good.
</Context>
<Query>
Tom is cute.
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer: yes, because the query can be found in the context 1.
</Answer>
</Question>
Pos:0, gold_answer:yes
Acc:	97%
Err:	0
1:	8/10
2:	10/10
3:	10/10
4:	10/10
5:	10/10
6:	10/10
7:	10/1

In [32]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. Bob is nice. 
2. Bob is good. 
3. Bob is funny.
Query: Bob is good.
Answer: yes, because the query can be found in the context 2.
</Example>
<Context>
{context}
</Context>
<Query>
{query}
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good."""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))

Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)



Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. Bob is nice. 
2. Bob is good. 
3. Bob is funny.
Query: Bob is good.
Answer: yes, because the query can be found in the context 2.
</Example>
<Context>
1. Tom is nice.
2. Tom is good.
</Context>
<Query>
Tom is cute.
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer: no, because the query is not mentioned in the given context.
</Answer>
</Exam>
Pos:0, gold_answer:yes
Acc:	28%
Err:	0
4:	1/10
5:	1/10
6:	5/10
7:	3/10
8:	6/10
9:	4/10
10:	8/10

Pos:0, gold_answer:no
Acc:	70%
Err:	0
0:	10/10
1:	10/10
2:	9/10
3:	6/10
4:	8/10
5:	5/10
6:	8/10
7:	6/10
8:	5/10
9:	3/10



In [33]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Context>
{context}
</Context>
<Query>
{query}
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good."""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))


Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Context>
1. Tom is nice.
2. Tom is good.
</Context>
<Query>
Tom is cute.
</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer: no, because the query can not be found in the contexts.
</Answer>
</Question>
Pos:0, gold_answer:yes
Acc:	78%
Err:	0
1:	8/10
2:	7/10
3:	5/10
4:	8/10
5:	9/10
6:	7/10
7:	7/10
8:	10/10
9:	8/10
10:	9/10

Pos:0, gold_answer:no
Acc:	51%
Err:	0
0:	5/10
1:	9/10
2:	9/10
3:	4/10
4:	3/10
5:	4/10
6:	7/10
7:	3/10
8:	4/10
9:	3/10



In [34]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
<Example 1>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example 1>
<Example 2>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example 2>
</Example>
<Context>
{context}</Context>
<Query>{query}</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good."""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))

Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
<Example 1>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example 1>
<Example 2>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example 2>
</Example>
<Context>
1. Tom is nice.
2. Tom is good.</Context>
<Query>Tom is cute.</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer: no, because the query can not be found in the contexts.
</Answer>
</Solution> 
Pos:0, gold_answer:yes
Acc:	71%
Err:	0
2:	3/10
3:	6/10
4:	9/10
5:	9/10
6:	9/10
7:	7/10
8:	10/10
9:	9/10
10:	9/10

Pos:0, gold_answer:no
Acc:	31%
Err:	0
0:	7/10
1:	10/10
2:	5/10
3:	1/10
4:	1/10
5:	1/10
6:	3/10
7:	1/10
8:

In [36]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Context>
{context}</Context>
<Query>{query}</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good."""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))


Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Context>
1. Tom is nice.
2. Tom is good.</Context>
<Query>Tom is cute.</Query>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer: no, because the query can not be found in the context.
</Answer>
</Question>
Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good

## 83-40

In [37]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind.
2. Fiona is smart. 
3. Chris is kind.
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind.
2. Fiona is smart. 
3. Chris is kind.
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Question>
Context:
{context}Query: {query}
</Question>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good.\n"""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))


Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: Determine if the query is mentioned in the given context.
If yes, please tell me the position of the query.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Question>
Context:
1. Tom is nice.
2. Tom is good.
Query:Tom is cute.
</Question>
<Format>
Please answer with the format:
Answer: <your answer here>
</Format>
<Answer>
Answer: no, because the query can not be found in the contexts.
</Answer>
</Question>
<Question>
Context:
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: John is smart.
</Question>
<Answer>
Answer: yes, because the query can be found in the contexts 1 and 3.
</Answer>
</Question>
<Question>
Context:
1. Tom is nice.
2

## 83-40 (best)

In [38]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Question>
Context:
{context}Query: {query}
</Question>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good.\n"""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))


Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


Task: Determine if the query is mentioned in the given context.
Please pay attention to spelling strictly,
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind, 
2. Fiona is smart, 
3. Chris is kind
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Question>
Context:
1. Tom is nice.
2. Tom is good.
Query: Tom is cute.
</Question>
<Answer>
Answer: no, because the query can not be found in the contexts.
</Answer>

Explanation:
In this task, you will be given a context and a query. Your task is to determine if the query is mentioned in the given context.

For each example, you will see a context and a query. You need to determine if the query is mentioned in the context. If the query is mentioned, you should answer "yes". If the query is not mentioned, you should answer "no".

For the first 

## 81-36

In [39]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""Task: Determine if the query is mentioned in the given context.
<Example>
Context: 
1. John is kind.
2. Fiona is smart.
3. Chris is kind.
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind.
2. Fiona is smart. 
3. Chris is kind.
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Question>
Context:
{context}Query: {query}
</Question>
<Answer>
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good.\n"""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))


Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


/home/yanghn/env_py/lib/python3.8/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: Determine if the query is mentioned in the given context.
<Example>
Context: 
1. John is kind.
2. Fiona is smart.
3. Chris is kind.
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
</Example>
<Example>
Context: 
1. John is kind.
2. Fiona is smart. 
3. Chris is kind.
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
</Example>
<Question>
Context:
1. Tom is nice.
2. Tom is good.
Query: Tom is cute.
</Question>
<Answer>
Answer: no, because the query can not be found in the contexts.
</Answer>
</Question>
</Task>

This task is designed to test the ability of the model to understand the context of a query and determine if it is mentioned in the given context. The model should be able to identify the context and determine if the query is present in it.

For example, in the first example, the query "Bob is good" is not mentioned in the context, so the answer should be "no". In the second example, the query "Fiona is 

## 28-20

In [40]:
def formulate_prompt_conclusion(context:str, query:str)->str:
    prompt = f"""'''Task''': Determine if the query is mentioned in the given context.
'''Example'''
Context: 
1. John is kind.
2. Fiona is smart.
3. Chris is kind.
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
'''Example'''
Context: 
1. John is kind.
2. Fiona is smart. 
3. Chris is kind.
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
'''Question'''
Context:
{context}Query: {query}
'''Answer'''
Answer:"""
    return prompt

context = """1. Tom is nice.
2. Tom is good.\n"""
query = "Tom is cute."

prompt = formulate_prompt_conclusion(context, query)
answer = ask(prompt)
print(answer)
# print(answer.replace(prompt, ''))


Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts']), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : data['num_fact'],
        'answer':'yes',
        'pos'   : 0,
    } for data in dataset['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev(prompt_answer_dict_list)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_conclusion(
            ablation_conclusion_formulate_factstr_factnum(
                data['facts'][1:]), 
                ablation_conclusion_parse_facts(data['facts'])[0]), 
        'num_fact'  : str(int(data['num_fact'])-1),
        'answer':'no',
        'pos'   : 0,
    } for data in dataset['train']]
evaluate_dev(prompt_answer_dict_list)


'''Task''': Determine if the query is mentioned in the given context.
'''Example'''
Context: 
1. John is kind.
2. Fiona is smart.
3. Chris is kind.
Query: Bob is good.
Answer: no, because the query can not be found in the contexts.
'''Example'''
Context: 
1. John is kind.
2. Fiona is smart. 
3. Chris is kind.
Query: Fiona is smart.
Answer: yes, because the query can be found in the contexts 2.
'''Question'''
Context:
1. Tom is nice.
2. Tom is good.
Query: Tom is cute.
'''Answer'''
Answer: no, because the query can not be found in the contexts.

This is a simple example of how a language model can be trained to answer questions based on the context of a given text. The model is trained on a dataset of text and corresponding answers, and learns to predict the answer to a given question based on the context of the text.

In this example, the model is trained to answer questions about the characteristics of people, and the context is provided by a set of statements about the people. The mo

# Compare with Exam

In [10]:
def formulate_prompt_comparison(context:str, query:str)->str:
    prompt = f"""Task: Determine if the two sentences are the same. Please pay attention to the spelling strictly.
<Example 1>
1. John is kind.
2. Fiona is smart. 
Answer: no
</Example 1>
<Example 2>
1. John is nice.
2. John is nice. 
Answer: yes
</Example 2>
<Sentence>
1: {context}
2: {query}
</Sentence>
<Answer>
Answer:"""
    return prompt
context = 'Bob is nice.'
query   = 'Bob is a nice'
prompt = formulate_prompt_comparison(context, query)
answer = ask(prompt)
print(answer)

/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yanghn/env_py/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Task: Determine if the two sentences are the same. Please pay attention to the spelling strictly.
<Example 1>
1. John is kind.
2. Fiona is smart. 
Answer: no
</Example 1>
<Example 2>
1. John is nice.
2. John is nice. 
Answer: yes
</Example 2>
<Sentence>
1: Bob is nice.
2: Bob is a nice
</Sentence>
<Answer>
Answer: no
</Answer>

Explanation:

In the first example, the two sentences are different because they have different words. "Kind" and "smart" are two different adjectives with different meanings.

In the second example, the two sentences are the same because they have the same words and the same meaning.

In the third example, the two sentences are different because they have different words and different meanings. "Nice" is an adjective in both sentences, but the second sentence also includes the word "a" which changes the meaning of the sentence.

Therefore, the answer to the task is "yes" for the second example and "no" for the first and third examples.


In [15]:
Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_100.jsonl"
dataset = load_dataset('json', data_files=path_dev)
dataset_fact2 = dataset.filter(lambda data: data['num_fact']==2)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_comparison(
            ablation_conclusion_parse_facts(data['facts'])[0],
            ablation_conclusion_parse_facts(data['facts'])[0],
        ), 
        'answer':'yes',
    } for data in dataset_fact2['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev_comparison(prompt_answer_dict_list)

prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_comparison(
            ablation_conclusion_parse_facts(data['facts'])[0],
            ablation_conclusion_parse_facts(data['facts'])[1],
        ), 
        'answer':'no',
    } for data in dataset_fact2['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev_comparison(prompt_answer_dict_list)


Task: Determine if the two sentences are the same. Please pay attention to the spelling strictly.
<Example 1>
1. John is kind.
2. Fiona is smart. 
Answer: no
</Example 1>
<Example 2>
1. John is nice.
2. John is nice. 
Answer: yes
</Example 2>
<Sentence>
1: Erin is round.
2: Erin is round.
</Sentence>
<Answer>
Answer:
[0] [correct]	Gold:[yes]; Ans:yes; yes
[1] [correct]	Gold:[yes]; Ans:yes; yes
[2] [correct]	Gold:[yes]; Ans:yes; yes
[3] [correct]	Gold:[yes]; Ans:yes; yes
[4] [correct]	Gold:[yes]; Ans:yes; yes
[5] [correct]	Gold:[yes]; Ans:yes; yes
[6] [correct]	Gold:[yes]; Ans:yes; yes
[7] [correct]	Gold:[yes]; Ans:yes; yes
[8] [correct]	Gold:[yes]; Ans:yes; yes
[9] [correct]	Gold:[yes]; Ans:yes; yes
gold_answer:yes
Acc:	10/10
Err:	0

Task: Determine if the two sentences are the same. Please pay attention to the spelling strictly.
<Example 1>
1. John is kind.
2. Fiona is smart. 
Answer: no
</Example 1>
<Example 2>
1. John is nice.
2. John is nice. 
Answer: yes
</Example 2>
<Sentence>
1:

In [20]:
Dataset.cleanup_cache_files
path_dev = "../dataset/fact/CWA_facts_same_diff_100.jsonl"
dataset_fact_same_diff = load_dataset('json', data_files=path_dev)
prompt_answer_dict_list = [
    {
        'prompt': formulate_prompt_comparison(
            data['fact_1'],
            data['fact_2'],
        ), 
        'answer':data['answer'],
    } for data in dataset_fact_same_diff['train']]
print(prompt_answer_dict_list[0]['prompt'])
evaluate_dev_comparison_same_diff(prompt_answer_dict_list)


Task: Determine if the two sentences are the same. Please pay attention to the spelling strictly.
<Example 1>
1. John is kind.
2. Fiona is smart. 
Answer: no
</Example 1>
<Example 2>
1. John is nice.
2. John is nice. 
Answer: yes
</Example 2>
<Sentence>
1: The rabbit eats the squirrel
2: The rabbit eats the squirrel
</Sentence>
<Answer>
Answer:
[0] [correct]	Gold:[yes]; Ans:['yes']; yes
[1] [correct]	Gold:[yes]; Ans:['yes']; yes
[2] [wrong]	Gold:[yes]; Ans:['no']; no
[3] [correct]	Gold:[yes]; Ans:['yes']; yes
[4] [correct]	Gold:[yes]; Ans:['yes']; yes
[5] [correct]	Gold:[yes]; Ans:['yes']; yes
[6] [correct]	Gold:[yes]; Ans:['yes']; yes
[7] [correct]	Gold:[yes]; Ans:['yes']; yes
[8] [correct]	Gold:[yes]; Ans:['yes']; yes
[9] [correct]	Gold:[yes]; Ans:['yes']; yes
[10] [correct]	Gold:[yes]; Ans:['yes']; yes
[11] [correct]	Gold:[yes]; Ans:['yes']; yes
[12] [correct]	Gold:[yes]; Ans:['yes']; yes
[13] [correct]	Gold:[yes]; Ans:['yes']; yes
[14] [correct]	Gold:[yes]; Ans:['yes']; yes
[15] [c